In [31]:
!pip install jieba

In [32]:
import requests
from bs4 import BeautifulSoup
import re
import jieba
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt

In [33]:
url = "https://zh.wikipedia.org/zh-tw/Seventeen%E6%BC%94%E5%94%B1%E6%9C%83%E5%88%97%E8%A1%A8"
headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"
}

res = requests.get(url, headers=headers)
soup = BeautifulSoup(res.text, "html.parser")

# 演唱會目錄

In [34]:
#《LIKE SEVENTEEN 'Boys Wish' Concert》
table1 = soup.find("table", class_="wikitable", style="text-align:center") 
rows1 = table1.find_all("tr")
data1 = []

for row in rows1:
    cols = row.find_all(["th", "td"])
    cols = [col.get_text(strip=True) for col in cols]
    data1.append(cols)

# 將數據傳換成 DataFrame
df1 = pd.DataFrame(data1[1:], columns=data1[0])

df1['國家／地區'] = df1['國家／地區'].fillna(method='ffill')
df1['城市'] = df1['城市'].fillna(method='ffill')
df1['舉行地點'] = df1['舉行地點'].fillna(method='ffill')

df1 = df1.drop('備註', axis=1)

df1

,日期,國家／地區,城市,舉行地點
0,2015年12月24日,南韓,首爾,龍山ART HALL
1,2015年12月25日,特別舞台 — Performance Team[2]，韓國時間14:00開始,首爾,龍山ART HALL
2,特別舞台 — Hip-hop Team[3]，韓國時間19:30開始,特別舞台 — Performance Team[2]，韓國時間14:00開始,首爾,龍山ART HALL
3,2015年12月26日,特別舞台 — UNIT HIGHLIGHTS,首爾,龍山ART HALL


In [45]:
tables = soup.find_all("table", class_="wikitable", style="text-align:center")

# 逐一處理每個表格
for i, table in enumerate(tables):
    # 提取表格的數據
    rows = table.find_all("tr")
    data = []

    for row in rows:
        cols = row.find_all(["th", "td"])
        cols = [col.get_text(strip=True) for col in cols]
        data.append(cols)

    # 將數據轉換成DataFrame
    df = pd.DataFrame(data[1:], columns=data[0])

    # 處理缺失值
    df = df.fillna(method='ffill')

    # 打印或使用 df 中的 DataFrame
    print(f"DataFrame {i + 1}:\n{df}\n")

DataFrame 1:
                                   日期                                   國家／地區  \
0                         2015年12月24日                                      南韓   
1                         2015年12月25日  特別舞台 — Performance Team[2]，韓國時間14:00開始   
2  特別舞台 — Hip-hop Team[3]，韓國時間19:30開始  特別舞台 — Performance Team[2]，韓國時間14:00開始   
3                         2015年12月26日                  特別舞台 — UNIT HIGHLIGHTS   

   城市        舉行地點                    備註  
0  首爾  龍山ART HALL  特別舞台 — Vocal Team[1]  
1  首爾  龍山ART HALL  特別舞台 — Vocal Team[1]  
2  首爾  龍山ART HALL  特別舞台 — Vocal Team[1]  
3  首爾  龍山ART HALL  特別舞台 — Vocal Team[1]  

DataFrame 2:
           日期                  國家／地區      城市                            舉行地點  \
0  2016年7月30日                     南韓      首爾                       首爾蠶室室內體育館   
1  2016年7月31日  特別舞台 — Dark SEVENTEEN      首爾                       首爾蠶室室內體育館   
2   2016年8月5日                     日本      大阪  大阪府立國際會議場（英語：Grand Cube Osaka）   
3   2016年8月8日                     東京 

In [68]:
final_df = pd.DataFrame(columns=['日期', '國家／地區', '城市', '舉行地點'])

# 逐一處理每個表格
for i, table in enumerate(tables):
    # 提取表格的數據
    rows = table.find_all("tr")
    data = []

    for row in rows:
        cols = row.find_all(["th", "td"])
        cols = [col.get_text(strip=True) for col in cols]
        data.append(cols)

    # 將數據轉換成DataFrame
    df = pd.DataFrame(data[1:], columns=data[0])

    # 使用正則表達式刪除被注解的部分
    df['日期'] = df['日期'].apply(lambda x: re.sub(r'\([^)]*\)', '', x))

    # 檢查表格是否包含 "舉行地點" 這一列
    if '舉行地點' not in df.columns:
        df['舉行地點'] = ''  # 如果不存在，則加入一列空值

    # 刪除沒有舉行地點的行
    df = df.dropna(subset=['舉行地點'])
    
    # 刪除 "備註 " 和 "主題" 這兩列
    if '備註' in df.columns:
        df = df.drop('備註', axis=1)
    if '主題' in df.columns:
        df = df.drop('主題', axis=1)

    # 遍歷表格的每一
    for index, columns in df.iterrows():
        # 如果 "國家／地區" 列是空白，則使用前面的非空白值進行填充
        if pd.isna(columns['國家／地區']):
            columns['國家／地區'] = final_df['國家／地區'].iloc[-1] if not final_df.empty else ''
        # 如果 "城市" 列是空白，則使用前面的非空白值進行填充
        if pd.isna(columns['城市']):
            columns['城市'] = final_df['城市'].iloc[-1] if not final_df.empty else ''
        # 如果 "舉行地點" 列是空白，則使用前面的非空白值進行填充
        if pd.isna(columns['舉行地點']):
            columns['舉行地點'] = final_df['舉行地點'].iloc[-1] if not final_df.empty else ''

    # 合併到最終 DataFrame
    final_df = pd.concat([final_df, df], ignore_index=True)

# 打印最終 DataFrame
print("Final DataFrame:\n", final_df)

Final DataFrame:
              日期 國家／地區      城市                             舉行地點
0   2015年12月24日    南韓      首爾                       龍山ART HALL
1    2016年7月30日    南韓      首爾                        首爾蠶室室內體育館
2     2016年8月5日    日本      大阪   大阪府立國際會議場（英語：Grand Cube Osaka）
3     2016年8月8日    東京  中野太陽廣場                                 
4    2017年2月15日    日本      神戶  神戶世界紀念館（英語：World Memorial Hall）
..          ...   ...     ...                              ...
68   2023年7月21日    南韓      首爾                           高尺天空巨蛋
69    2023年9月6日    日本      東京                             東京巨蛋
70  2023年12月23日    泰國      曼谷                         拉加曼加拉體育場
71   2024年1月13日   菲律賓     布拉干                           菲律賓體育館
72   2024年1月20日    澳門      澳門                         奧林匹克體育中心

[73 rows x 4 columns]


In [117]:
##修正3
final_df = pd.DataFrame(columns=['日期', '國家／地區', '城市', '舉行地點'])

# 逐一處理每個表格
for i, table in enumerate(tables):
    # 檢查索引是否在需要刪除的範圍內
    if i + 1 in [11, 12, 13]:
        continue  # 跳過不處理

    # 提取表格的數據
    rows = table.find_all("tr")
    data = []

    for row in rows:
        cols = row.find_all(["th", "td"])
        cols = [col.get_text(strip=True) for col in cols]
        data.append(cols)
        

    # 將數據轉換成 DataFrame
    df = pd.DataFrame(data[1:], columns=data[0])
    
    # 刪除 "備註 "這列
    if '備註' in df.columns:
        df = df.drop(columns=['備註'])
    
    
    
    print(f"DataFrame {i + 1}:\n{df}\n")

DataFrame 1:
                                   日期                                   國家／地區  \
0                         2015年12月24日                                      南韓   
1                         2015年12月25日  特別舞台 — Performance Team[2]，韓國時間14:00開始   
2  特別舞台 — Hip-hop Team[3]，韓國時間19:30開始                                    None   
3                         2015年12月26日                  特別舞台 — UNIT HIGHLIGHTS   

     城市        舉行地點  
0    首爾  龍山ART HALL  
1  None        None  
2  None        None  
3  None        None  

DataFrame 2:
           日期                  國家／地區      城市                            舉行地點
0  2016年7月30日                     南韓      首爾                       首爾蠶室室內體育館
1  2016年7月31日  特別舞台 — Dark SEVENTEEN    None                            None
2   2016年8月5日                     日本      大阪  大阪府立國際會議場（英語：Grand Cube Osaka）
3   2016年8月8日                     東京  中野太陽廣場                                
4   2016年8月9日                舉辦日夜兩場次    None                            

In [ ]:
# 合併到最終 DataFrame
    final_df = pd.concat([final_df, df], ignore_index=True)

# 打印最終 DataFrame
print("Final DataFrame:\n", final_df)

In [99]:
# 計算城市次數
city_counts = final_df['國家／地區'].value_counts(dropna=False)

# 打印城市次數
print("國家／地區次數:\n", city_counts)

國家／地區次數:
 日本                                        19
                                          14
南韓                                        12
大阪                                        10
菲律賓                                        7
印度尼西亞                                      6
美國                                         6
東京                                         5
泰國                                         5
福岡                                         4
埼玉                                         4
千葉                                         4
愛知                                         4
臺灣                                         3
新加坡                                        3
加拿大                                        3
馬來西亞                                       3
橫濱                                         3
特別舞台 — Performance Team[2]，韓國時間14:00開始     2
芝加哥                                        2
休斯頓                                        2
靜岡                                         2


# 製作圖表


In [ ]:
for place, info in places.items():
    # 使用 place 作為鍵來檢索重複次數
    count = word_freq.get(place, 0)  # 如果未找到該景點，默認重複次數為 0
    if count > 0:  # 只有當重複次數大於0時才添加標記
        color = 'green' if count <= 5 else 'orange' if count <= 10 else 'red'

        folium.Marker(
            location=[info['lat'], info['lon']],
            popup=f"{place}<br>出現次數: {count}",
            icon=folium.Icon(color=color),
        ).add_to(m)

# 顯示地圖
m